Q5)For all the body styles involved in crashes, mention the top ethnic user group of each unique body style 

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [0]:
df_prim_person=spark.read.format("csv").option("inferschema",True).option("header",True).load("dbfs:/FileStore/Primary_Person_use.csv")

In [0]:
df_units=spark.read.format("csv").option("inferschema",True).option("header",True).load("dbfs:/FileStore/Units_use.csv")

A single person in the primary_person dataset is uniquely  identified by (CRASH_ID , UNIT_NBR , PRSN_NBR)

In [0]:
df_prim_person.select("PRSN_NBR").distinct().show()

+--------+
|PRSN_NBR|
+--------+
|       1|
+--------+



As PRSN_NBR is same for all records in primary_person dataset, we can uniquely identify each person just by (CRASH_ID , UNIT_NBR )

In [0]:
df_charges=spark.read.format("csv").option("inferschema",True).option("header",True).load("dbfs:/FileStore/Charges_use.csv")

In [0]:
df_prim_person.join(df_units,(df_prim_person.CRASH_ID==df_units.CRASH_ID) 
 & (df_prim_person.UNIT_NBR==df_units.UNIT_NBR)  ).select(df_units.CRASH_ID,df_units.UNIT_NBR,"VEH_BODY_STYL_ID","PRSN_ETHNICITY_ID").show()

+--------+--------+--------------------+-----------------+
|CRASH_ID|UNIT_NBR|    VEH_BODY_STYL_ID|PRSN_ETHNICITY_ID|
+--------+--------+--------------------+-----------------+
|14768622|       1|PASSENGER CAR, 4-...|         HISPANIC|
|14838637|       1|PASSENGER CAR, 4-...|            WHITE|
|14838637|       2|               TRUCK|            BLACK|
|14838637|       2|               TRUCK|            BLACK|
|14838641|       1|PASSENGER CAR, 4-...|            WHITE|
|14838641|       2|SPORT UTILITY VEH...|            BLACK|
|14838668|       1|PASSENGER CAR, 4-...|         HISPANIC|
|14838669|       1|              PICKUP|            WHITE|
|14838669|       2|PASSENGER CAR, 4-...|            WHITE|
|14838670|       1|SPORT UTILITY VEH...|            WHITE|
|14838685|       1|PASSENGER CAR, 4-...|            BLACK|
|14838693|       1|              PICKUP|         HISPANIC|
|14838768|       1|SPORT UTILITY VEH...|            ASIAN|
|14838768|       2|PASSENGER CAR, 4-...|         HISPANI

In [0]:
df_temp1=df_prim_person.join(df_units,(df_prim_person.CRASH_ID==df_units.CRASH_ID) 
 & (df_prim_person.UNIT_NBR==df_units.UNIT_NBR)  ).select(df_units.CRASH_ID,df_units.UNIT_NBR,"VEH_BODY_STYL_ID","PRSN_ETHNICITY_ID")

Removing undesired values of 'vehicle body style' and 'ethinicity'

In [0]:
df_temp2=df_temp1.filter((col("VEH_BODY_STYL_ID")!="NA") & (col("PRSN_ETHNICITY_ID")!="NA")).filter((col("VEH_BODY_STYL_ID")!="UNKNOWN") & (col("PRSN_ETHNICITY_ID")!="UNKNOWN"))

In [0]:
df_temp2.groupBy("VEH_BODY_STYL_ID","PRSN_ETHNICITY_ID").count().show()

+--------------------+--------------------+-----+
|    VEH_BODY_STYL_ID|   PRSN_ETHNICITY_ID|count|
+--------------------+--------------------+-----+
|      FARM EQUIPMENT|               WHITE|   27|
|       TRUCK TRACTOR|               BLACK| 1130|
|       TRUCK TRACTOR|               ASIAN|   64|
|                 VAN|AMER. INDIAN/ALAS...|    9|
|          MOTORCYCLE|               ASIAN|    2|
|                 BUS|               BLACK|  157|
|                 BUS|               WHITE|  123|
|SPORT UTILITY VEH...|               ASIAN| 1018|
|                 VAN|            HISPANIC| 2029|
|           AMBULANCE|            HISPANIC|   11|
|OTHER  (EXPLAIN I...|               ASIAN|    5|
|                 VAN|               BLACK|  691|
|SPORT UTILITY VEH...|            HISPANIC|10619|
|                 BUS|            HISPANIC|  158|
|              PICKUP|               BLACK| 2443|
|              PICKUP|AMER. INDIAN/ALAS...|   31|
|          MOTORCYCLE|               BLACK|   75|


In [0]:
df_temp3=df_temp2.groupBy("VEH_BODY_STYL_ID","PRSN_ETHNICITY_ID").count()

In [0]:
window=Window.partitionBy("VEH_BODY_STYL_ID").orderBy(desc("count"))

In [0]:
df_temp3.withColumn("rank",dense_rank().over(window)).show()

+----------------+-----------------+-----+----+
|VEH_BODY_STYL_ID|PRSN_ETHNICITY_ID|count|rank|
+----------------+-----------------+-----+----+
|       AMBULANCE|            WHITE|   55|   1|
|       AMBULANCE|         HISPANIC|   11|   2|
|       AMBULANCE|            BLACK|    8|   3|
|       AMBULANCE|            OTHER|    4|   4|
|       AMBULANCE|            ASIAN|    1|   5|
|             BUS|         HISPANIC|  158|   1|
|             BUS|            BLACK|  157|   2|
|             BUS|            WHITE|  123|   3|
|             BUS|            ASIAN|   11|   4|
|             BUS|            OTHER|    4|   5|
|  FARM EQUIPMENT|            WHITE|   27|   1|
|  FARM EQUIPMENT|         HISPANIC|   19|   2|
|  FARM EQUIPMENT|            BLACK|    7|   3|
|      FIRE TRUCK|            WHITE|   64|   1|
|      FIRE TRUCK|            BLACK|    9|   2|
|      FIRE TRUCK|         HISPANIC|    4|   3|
|      FIRE TRUCK|            OTHER|    2|   4|
|      MOTORCYCLE|            WHITE|  49

In [0]:
df_temp4=df_temp3.withColumn("rank",dense_rank().over(window))

In [0]:
display(df_temp4.filter(col("rank")==1).select("VEH_BODY_STYL_ID","PRSN_ETHNICITY_ID"))

VEH_BODY_STYL_ID,PRSN_ETHNICITY_ID
AMBULANCE,WHITE
BUS,HISPANIC
FARM EQUIPMENT,WHITE
FIRE TRUCK,WHITE
MOTORCYCLE,WHITE
NEV-NEIGHBORHOOD ELECTRIC VEHICLE,WHITE
NOT REPORTED,WHITE
OTHER (EXPLAIN IN NARRATIVE),WHITE
"PASSENGER CAR, 2-DOOR",WHITE
"PASSENGER CAR, 4-DOOR",WHITE
